In [1]:
import keras as k
import scipy
import numpy as np
import pandas as pd

Using Theano backend.


In [2]:
import numpy as np
import pandas as pd

import pickle


In [3]:
def dumpfiles(home, fsize, dp, timestep, phi=0.1, cohesion=''):
    if dp == 75:
        home = home + '/alpp' + str(phi) + cohesion +'/CFD/' + str(timestep) + '/'
    else:
        home = home + '/alpp' + str(phi) + 'dp' + str(dp) + 'mm' + cohesion + '/CFD/' + str(timestep) + '/'
    alpp_file = 'barAlpp' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    alpp = pd.read_csv(home+alpp_file,skiprows=21,skipfooter=33)
    numpy_alpp = alpp.values

    Us_file = 'tildeUs' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    Us = pd.read_csv(home+Us_file,skiprows=21,skipfooter=33)
    Us = Us.values
    numpy_Us = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in Us])

    Uf_file = 'tildeUf' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    Uf = pd.read_csv(home+Uf_file,skiprows=21,skipfooter=33)
    Uf = Uf.values
    numpy_Uf = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in Uf])

    numpy_Uslip = numpy_Uf - numpy_Us

    betaNum_file = 'normalizedEulerianDragCoeffNum' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    betaNum = pd.read_csv(home+betaNum_file,skiprows=19,skipfooter=4)
    betaNum = betaNum.values
    numpy_betaNum = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in betaNum])

    betaDom_file = 'normalizedEulerianDragCoeffDom' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    betaDom = pd.read_csv(home+betaDom_file,skiprows=19,skipfooter=4)
    betaDom = betaDom.values
    numpy_betaDom = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in betaDom])

    numpy_ratio = numpy_betaNum / numpy_betaDom

    original_table = np.concatenate((numpy_alpp,numpy_Uslip ),axis=1)

    nx = 80
    ny = 80
    nz = 320
    dx = 1

    index1=0
    npData=4

    myData = np.zeros( ((nx-2*fsize)*(ny-2*fsize)*(nz-2*fsize), npData * (2*dx +1)**3) );
    target = np.zeros((nx-2*fsize)*(ny-2*fsize)*(nz-2*fsize))
    for k in range(fsize,nz-fsize):
        for j in range(fsize,ny-fsize):
            for i in range(fsize,nx-fsize):
                target[index1] = numpy_ratio[k*ny*nx+j*nx + i,-1]
                index2 = 0;
                for ii in np.array(range(-dx,dx+1))*fsize:
                    for jj in np.array(range(-dx,dx+1))*fsize:
                        for kk in np.array(range(-dx,dx+1))*fsize:
                            stencil= i+ii + (j+jj)*nx + (k+kk)*nx*ny;
                            #target[index1] = numpy_ratio[stencil,-1]
                            temp = original_table[stencil][:]
                            for ind in range(npData):
                                myData[index1,index2] = temp[ind];
                                index2 = index2 + 1
                index1 = index1 +1   
    myData = np.hstack((np.ones((myData.shape[0],1))*(1/(3*fsize)),myData))
    file_myData = "coarse_corrected_myData"+'fsize' + str(fsize)+'phi'+str(phi)+'dp'+str(dp)+'timestep'+str(timestep)+cohesion+".pickle"
    file_target = "coarse_corrected_target"+'fsize' + str(fsize)+'phi'+str(phi)+'dp'+str(dp)+'timestep'+str(timestep)+cohesion+".pickle"
    pickle.dump( myData, open( file_myData, "wb" ) )
    pickle.dump( target, open( file_target, "wb" ) )
    return myData

In [4]:
home = '/home/nandhini/aozel/periodicBox'

In [5]:
np.array(range(5))*5

array([ 0,  5, 10, 15, 20])

In [6]:
dumpfiles(home=home, dp = 75, timestep=0.625, fsize=9)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03703704,  0.230587  , -0.0086263 , ..., -0.070758  ,
         0.32467   ,  0.2527    ],
       [ 0.03703704,  0.206693  , -0.0127582 , ..., -0.041225  ,
         0.357411  ,  0.25337   ],
       [ 0.03703704,  0.184211  , -0.011387  , ..., -0.024309  ,
         0.379778  ,  0.288913  ],
       ..., 
       [ 0.03703704,  0.126365  , -0.074374  , ..., -0.0033628 ,
        -0.014422  ,  0.2751854 ],
       [ 0.03703704,  0.11769   , -0.07724   , ..., -0.0031229 ,
        -0.036414  ,  0.3293481 ],
       [ 0.03703704,  0.109757  , -0.07738   , ..., -0.0028622 ,
        -0.062837  ,  0.3788935 ]])

In [7]:
dumpfiles(home=home, dp = 75, timestep=0.6, fsize=9)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03703704,  0.114923  , -0.1191323 , ..., -0.057862  ,
        -0.1197702 ,  0.291328  ],
       [ 0.03703704,  0.124813  , -0.1114868 , ..., -0.04323   ,
        -0.1658468 ,  0.354388  ],
       [ 0.03703704,  0.140511  , -0.102522  , ..., -0.02952   ,
        -0.2140763 ,  0.410842  ],
       ..., 
       [ 0.03703704,  0.124654  , -0.0093858 , ..., -0.0780957 ,
        -0.008342  ,  0.2064529 ],
       [ 0.03703704,  0.124777  , -0.0013003 , ..., -0.0954066 ,
         0.012342  ,  0.1889113 ],
       [ 0.03703704,  0.124716  ,  0.0077737 , ..., -0.1013464 ,
         0.017948  ,  0.1816744 ]])

In [8]:
dumpfiles(home=home, dp = 75, timestep=0.7, fsize=9)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03703704,  0.0582802 ,  0.094314  , ...,  0.0649755 ,
         0.062222  ,  0.548808  ],
       [ 0.03703704,  0.0474097 ,  0.09588   , ...,  0.0332442 ,
         0.074784  ,  0.520395  ],
       [ 0.03703704,  0.0439463 ,  0.083854  , ...,  0.0057229 ,
         0.086508  ,  0.5021791 ],
       ..., 
       [ 0.03703704,  0.0222066 , -0.215335  , ...,  0.072252  ,
        -0.04714   ,  0.2628945 ],
       [ 0.03703704,  0.0230297 , -0.205018  , ...,  0.075174  ,
        -0.045654  ,  0.2771941 ],
       [ 0.03703704,  0.0268985 , -0.191544  , ...,  0.074278  ,
        -0.049834  ,  0.2916091 ]])

In [9]:
dumpfiles(home=home, dp = 75, timestep=0.525, fsize=5)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.06666667,  0.00961373,  0.354591  , ...,  0.0615098 ,
         0.035254  ,  0.190559  ],
       [ 0.06666667,  0.00772786,  0.4121811 , ...,  0.0633542 ,
         0.021586  ,  0.154867  ],
       [ 0.06666667,  0.00739368,  0.4522285 , ...,  0.048813  ,
         0.02662   ,  0.174492  ],
       ..., 
       [ 0.06666667,  0.403978  , -0.0196353 , ...,  0.382031  ,
        -0.0034673 ,  0.616264  ],
       [ 0.06666667,  0.34984   , -0.0213037 , ...,  0.410277  ,
        -0.0648016 ,  0.669441  ],
       [ 0.06666667,  0.27485   ,  0.0027843 , ...,  0.462399  ,
        -0.112788  ,  0.725422  ]])

In [10]:
dumpfiles(home=home, fsize=11, phi=0.1, dp = 75, timestep=0.6, cohesion='')

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03030303,  0.128717  , -0.0749342 , ...,  0.0181596 ,
        -0.01421   ,  0.5041056 ],
       [ 0.03030303,  0.141367  , -0.0780457 , ...,  0.036606  ,
        -0.009898  ,  0.5521645 ],
       [ 0.03030303,  0.15204   , -0.0771852 , ...,  0.0533389 ,
        -0.00127   ,  0.5869266 ],
       ..., 
       [ 0.03030303,  0.174937  ,  0.0159567 , ..., -0.0647433 ,
        -0.020179  ,  0.241384  ],
       [ 0.03030303,  0.172694  ,  0.0094184 , ..., -0.0712214 ,
        -0.000952  ,  0.2085918 ],
       [ 0.03030303,  0.170591  ,  0.002817  , ..., -0.0772145 ,
         0.013073  ,  0.183299  ]])

In [11]:
dumpfiles(home=home, dp = 75, timestep=0.55, fsize=11)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03030303,  0.2223    , -0.031492  , ..., -0.0202239 ,
         0.0291037 ,  0.0652781 ],
       [ 0.03030303,  0.224511  , -0.022484  , ..., -0.0192515 ,
         0.0265676 ,  0.0656268 ],
       [ 0.03030303,  0.221862  , -0.0179641 , ..., -0.0177232 ,
         0.0253244 ,  0.0675742 ],
       ..., 
       [ 0.03030303,  0.029731  , -0.196301  , ..., -0.0692438 ,
         0.0015691 ,  0.0778637 ],
       [ 0.03030303,  0.0292404 , -0.232076  , ..., -0.05278357,
        -0.0014087 ,  0.07824342],
       [ 0.03030303,  0.0301711 , -0.27459   , ..., -0.038886  ,
        -0.0028511 ,  0.072438  ]])

In [12]:
dumpfiles(home=home, dp = 150, timestep=0.6, fsize=9)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03703704,  0.0698447 ,  0.1437161 , ...,  0.035647  ,
        -0.1694075 ,  1.094199  ],
       [ 0.03703704,  0.0779965 ,  0.13907313, ...,  0.0433859 ,
        -0.1645355 ,  1.075394  ],
       [ 0.03703704,  0.0854055 ,  0.128037  , ...,  0.05148814,
        -0.1616661 ,  1.047913  ],
       ..., 
       [ 0.03703704,  0.0239554 , -0.344855  , ...,  0.04649728,
         0.1291158 ,  1.20989   ],
       [ 0.03703704,  0.0271261 , -0.326684  , ...,  0.06992359,
         0.1326442 ,  1.123998  ],
       [ 0.03703704,  0.0306656 , -0.30749   , ...,  0.07945092,
         0.1406738 ,  1.05465   ]])

In [13]:
dumpfiles(home=home, dp = 300, timestep=0.65, fsize=9)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

array([[ 0.03703704,  0.0421904 , -0.8809473 , ..., -0.2320782 ,
         0.4740697 ,  2.49543   ],
       [ 0.03703704,  0.0379718 , -0.9577835 , ..., -0.2342683 ,
         0.4487102 ,  2.355529  ],
       [ 0.03703704,  0.0353677 , -1.0089139 , ..., -0.2254859 ,
         0.40496027,  2.147869  ],
       ..., 
       [ 0.03703704,  0.03029   ,  0.16992   , ..., -0.6021199 ,
         0.416458  ,  2.489093  ],
       [ 0.03703704,  0.0296433 ,  0.194342  , ..., -0.6665588 ,
         0.360399  ,  2.827832  ],
       [ 0.03703704,  0.0319479 ,  0.204868  , ..., -0.744278  ,
         0.289237  ,  3.131793  ]])

In [ ]:
#dumpfiles(home=home, dp = 150, timestep=0.62, fsize=11)

In [ ]:
dumpfiles(home=home, dp = 75, timestep=0.55, fsize=13)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/hema/yile/anaconda3/lib/python3

In [ ]:
dumpfiles(home=home, dp = 300, timestep=0.525, fsize=9)

In [ ]:
dumpfiles(home=home, dp = 150, timestep=0.65, fsize=11)

In [ ]:
dumpfiles(home=home, dp = 75, timestep=0.65, fsize=9)

In [ ]:
dumpfiles(home=home, dp = 150, timestep=0.55, fsize=9)

In [ ]:
dumpfiles(home=home, dp = 300, timestep=0.6, fsize=9)

In [ ]:
dumpfiles(home=home, dp = 150, timestep=0.525, fsize=11)

In [ ]:
dumpfiles(home=home, dp = 75, timestep=0.65, fsize=13)

In [ ]:
dumpfiles(home=home, dp = 75, timestep=0.325, fsize=13,phi=0.3, cohesion='HighCoh')